## 准备数据

In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [5]:
class MyModel(nn.Module):
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(784, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 10)
        
    def forward(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x = x.view(-1, 784)
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.layer3(x)

model = MyModel()

optimizer = optim.Adam(model.parameters())

## 计算 loss

In [7]:
import torch
from torch import optim
from torch.nn import functional as F

# 定义损失计算
def compute_loss(logits, labels):
    return F.cross_entropy(logits, labels)

# 定义准确率计算
def compute_accuracy(logits, labels):
    predictions = logits.argmax(dim=1, keepdim=True).squeeze()
    correct = predictions.eq(labels.view_as(predictions)).sum().item()
    accuracy = correct / len(labels)
    return accuracy

# 定义单步训练过程
def train_one_step(model, optimizer, x, y):
    model.train()
    optimizer.zero_grad()
    logits = model(x)
    loss = compute_loss(logits, y)
    loss.backward()
    optimizer.step()
    accuracy = compute_accuracy(logits, y)
    return loss.item(), accuracy

# 定义测试过程
def test(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in data_loader:
            logits = model(x)
            test_loss += compute_loss(logits, y).item()
            correct += (logits.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= len(data_loader.dataset)
    accuracy = correct / len(data_loader.dataset)
    return test_loss, accuracy


In [8]:
# 训练循环
for epoch in range(50):
    model.train()  # 将模型设置为训练模式
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)  # 如果使用GPU
        loss, accuracy = train_one_step(model, optimizer, X, y)
        total_loss += loss
        total_correct += accuracy * X.size(0)
        total_samples += X.size(0)
    print(f'Epoch {epoch}: Loss {total_loss / total_samples}, Accuracy {total_correct / total_samples}')

# 测试循环
model.eval()  # 将模型设置为评估模式
test_loss, test_accuracy = test(model, test_loader)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

ValueError: too many values to unpack (expected 2)

In [6]:
loss_fn = nn.CrossEntropyLoss()

def train_one_epoch(model, optimizer, data_loader):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()
        logits = model(X)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()

def test(model, data_loader):
    model.eval()
    total_loss = 0
    correct = 0
    with torch.no_grad():
        for X, y in data_loader:
            logits = model(X)
            total_loss += loss_fn(logits, y).item()
            correct += (logits.argmax(1) == y).type(torch.float).sum().item()
    avg_loss = total_loss / len(data_loader.dataset)
    accuracy = correct / len(data_loader.dataset)
    return avg_loss, accuracy

train_loader, test_loader = mnist_dataset()

for epoch in range(50):
    train_one_epoch(model, optimizer, train_loader)

avg_loss, accuracy = test(model, test_loader)
print('Test set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(avg_loss, accuracy*100))

ValueError: too many values to unpack (expected 2)